In [1]:
using DifferentialEquations, Plots

In [7]:
function Immune!(du,u,h,p,t)
    
    hist1 = h(p, t-tau)[1] #V
    hist2 = h(p, t-tau)[2] #S
    hist3 = h(p, t-tau)[3] #I
    hist4 = h(p, t-tau)[4] #R
    ###Secondary
    hist5 = h(p, t-taubm)[1] #V
    hist6 = h(p, t-taubm)[10] #M
    ###
    du[1] = phat*u[3]-dV*u[1]-n*(Emax*u[1]*(u[13]^h2))/((Ic)^h2+u[13]^h2);#V,secondary infection  
    du[2] = lamS*(1-(u[2]+u[3]+u[4]+u[5])/Smax)*u[2]-beta*u[2]*u[1];#S
    du[3] = (beta/(1+u[7]/eps_F_I))*hist1*hist2*u[8]-dI*u[3];#I
    du[4] = lamS*(1-(u[2]+u[3]+u[4]+u[5])/Smax)*u[4]+(beta*hist1*hist2)*u[9]/(1+eps_F_I/u[7]);#R
    du[5] = dI*u[3]-dD*u[5];#D
    du[6] = psi_F_prod+p_F_I*u[3]/(u[3]+eta_F_I)-k_lin_F*u[6]-k_B_F*((T_star+u[3])*A_F-u[7])*u[6]+k_U_F*u[7];#FU
    du[7] = -k_int_F*u[7]+k_B_F*((T_star+u[3])*A_F-u[7])*u[6]-k_U_F*u[7];#FB
    du[8] = u[8]*delta*(hist3-u[3])#A1
    du[9] = u[9]*delta*(hist4-u[4])#A2
    ###Secondary Immune Response:
    du[10] = rho_m*(2*Pb-1)*hist5*hist6-delta_m*u[10]#M
    du[11] = (1-Pb)*Pp*rho_m*u[10]-delta_b*u[11];#B 
    du[12] = (1-Pb)*(1-Pp)*rho_m*u[10]-delta_p*u[12];#P
    du[13] = a_p*u[12]-delta_a*u[13];#A  
end

Immune! (generic function with 1 method)

In [64]:
tspan = (0.0,600);
#####################################
n=0.5;
beta = 0.3;eta_F_I=0.022328;
tau= 0.17;Smax = 0.16;lamS = 0.74; dI = 0.1; dD=8;phat =394;dV = 8.4;
psi_F_prod=0.25;k_lin_F=16.635;k_B_F=0.0107;
T_star=1.104*1e-4;k_U_F=6.072;k_int_F=16.968;R_F_T = 1000;R_F_I = 1300;MM_F = 19000;avo=6.02214E23;
A_F = (MM_F/avo)*(R_F_I+R_F_T)*(1/5000)*(10^9*1e12);delta = 0.1;
###Humoral Immunity
mu_b=3.9E-6;#Original
delta_bn=0.0124;
taub=5;rho=0.0046;
alpha_v=0.1;alpha_IL4=0.1;
S_IL4=1E+4;
mu_t=0.0355;delta_t=0.055;
mu_IL4=1.3;delta_IL4=0.0132;
#####Secondary
taubm=3;
rho_m=0.0266;
delta_m=0.061;
delta_a=0.034;
Pb=0.1;
Pp=0.9;
delta_b=0.0322;
delta_p=0.01;
delta_ps=0.15
rhos=2
####*****************************************************************Strong,Delta
# a_b=0.0001*20;a_p=0.0001*20;#Minimum,Used
# a_ps=0.8*20;a_p=0.8*20;#Mean,Used
a_ps=0.8*20;a_p=0.8*20;#Mean,Used
# a_b=200*20;a_p=200*20;#Maximum,Used
###############
#Wuhan
#90 
# B0 = 19466.123268517622
# P0 = 14485.351801742989
# M0 = 2715.2898176532435
# A0 = 365186.74236199015
# ###############
# #180
# B0 = 1060.3572584705707
# P0 = 7031.597925300424
# M0 = 2950.737888943075
# A0 = 217886.5251673358
# ###
#360
B0 = 3.217618113595361
PS0 = 4.954447777944039e-12
P0 = 1191.163838139992
M0 = 2964.318262058176
A0 = 39607.72712179518
####*********************************************
#Omicron
#90
# B0 = 14726.352008568016
# P0 = 11831.682870443467
# M0 = 2253.5393837183487
# A0 = 299710.9615240191
# ###############
# #180
# B0 = 803.0184763342228
# P0 = 5674.664088849107
# M0 = 2431.666895220184
# A0 = 176400.02416013
# # ###
# # #360
# B0 = 2.4376884225716
# P0 = 959.8677464595415
# M0 = 2441.953357472441
# A0 = 31920.990138457935
###############################################
V0=4.5;
# V0=10;
u0=[V0; 0.16;0.0 ;0.0 ; 0.0;0.015 ;1.1e-8 ;1.0 ;1.0;M0;B0;P0;A0];#1
# u0=[V0; 0.16;0.0 ;0.0 ; 0.0;0.015 ;1.1e-8 ;1.0 ;1.0;M0;B0;PS0;P0;A0];#2
#####################################################################
p_F_I=2.8235*1e4*1e-4;eps_F_I =2*1e-4;
###
lags = [tau,taubm];

In [65]:
#Secondary
h(p,t) = [V0; 0.16;0.0 ;0.0 ; 0.0;0.015 ;1.1e-8 ;1.0 ;1.0;M0;B0;P0;A0];#1
# h(p,t) = [V0; 0.16;0.0 ;0.0 ; 0.0;0.015 ;1.1e-8 ;1.0 ;1.0;M0;B0;PS0;P0;A0];#2
alg = MethodOfSteps(Tsit5())
##
Emax=99;h2=1.2;Ic=662#Wuhan
# Emax=98.7;h2=0.79;Ic=5893.32#Omicron
###Secondary:
p1=[tau,phat,dV,beta,Smax,lamS,dI,eps_F_I,dD,psi_F_prod, p_F_I,eta_F_I,k_lin_F,k_B_F,T_star,k_U_F,k_int_F,A_F,
k_B_F,delta,mu_b,delta_bn,taub,rho,Pb,Pp,alpha_v,alpha_IL4,S_IL4,delta_b,delta_p,delta_m,a_ps,a_p,
delta_a,mu_t,delta_t,mu_IL4,delta_IL4,rho_m,Emax,h2,Ic,rho_m,delta_m,taubm];
###
z1= DDEProblem(Immune!,u0,h,tspan,p1;constant_lags=lags);
sol1 = solve(z1,saveat=0:0.01:600,alg,abstol=1e-8,reltol=1e-8);